Decision Tree workflow/process 
    Select feature
    Sort data
    Compute mid point or threshold value 
    Split (feature based on threshold)
    Compute fid3
    Compute entropy
    Compute weighted entropy
    Repeate for all threshold values
    Repeate for all features
    Find the weighted entropy with least value => best split 




In [37]:
#Import python libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


In [38]:
#Create a dummy dataset
array_x = np.array([[3,7],[1,8],[4,5],[2,6]])
print(array_x)

array_y = np.array([1, 0, 1, 0])
print(array_y)

[[3 7]
 [1 8]
 [4 5]
 [2 6]]
[1 0 1 0]


In [39]:
#Class Node (Root node, internal/decision node and leaf node)
class Node:
    def __init__(self,  feature = None, threshold = None, left = None, right = None, value = None):
        self.feature  = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value 

    def is_leaf_node(self):
        return self.value is not None
    

In [40]:
#Common methods in decision trees
def most_common_label(y):
    labels, counts = np.unique(y, return_count = True)
    return labels[np.argmax(counts)]

def split(x_column, split_threshold):
    left_node = np.argwhere(x_column <= split_threshold).flatten()
    right_node = np.argwhere(x_column > split_threshold).flatten()
    return left_node, right_node;

def entropy(y):
    fid3 = np.mean(y)
    if fid3 == 0 or fid3 == 1:
        return 0
    
    return -fid3 * np.log(fid3) - (1 - fid3) * np.log(1 - fid3)

def information_gain(x_column, y, split_threshold):
    left_node, right_node = split(x_column, split_threshold)
    if (len(left_node) == 0 or len(right_node) == 0):
        return 0
    
    p_left = len(left_node)/len(y)
    p_right = len(right_node)/len(y)
    weighted_entropy = p_left * entropy(y[left_node]) - p_right * entropy(y[right_node])
    return weighted_entropy


In [ ]:
#Follow the steps and compute weighted gain of all threshold for one feature using above methods
best_gain = float("inf") #this gives infinite number/value
split_feature =None
split_threshold = None

#Select the feature
x_column = array_x[ :, 0]
# print(x_column)

#Sort the data
x_column_sorted = np.sort(x_column)
# print(x_column_sorted)

#Calcualte mid point/threshold values
mid_col_1 = x_column_sorted[: -1]
# print(mid_col_1)
mid_col_2 = x_column_sorted[1: ]
# print(mid_col_2)

threshold = (mid_col_1 + mid_col_2)/2
# print(threshold)

for th in threshold:
  weighted_entropy = information_gain(x_column, array_y, th)
  print(f"Threshold {th} : {weighted_entropy}")

  if(weighted_entropy < best_gain):
    best_gain = weighted_entropy
    split_feature = 0
    split_threshold = th

print(f"\nbest_gain : {best_gain} \nsplit_feature : {split_feature} \nsplit_threshold : {split_threshold}")


Threshold 1.5 : -0.4773856262211096
Threshold 2.5 : 0.0
Threshold 3.5 : 0.4773856262211096

best_gain : -0.4773856262211096 
split_feature : 0 
split_threshold : 1.5


In [ ]:
#Follow the steps and compute weighted gain of all threshold for all features
#The one with the least weighted gain is the best split
#The method best_split will take input and output varaibles and returns the best split (feature and threshold) which will be used for spliting 
def best_split(x, y):
  num_sample, num_feature = x.shape
  best_gain = float("inf")
  split_feature = None
  split_threshold = None

  for feature in range(num_feature):
    x_column = x[ : , feature]
    x_column_sorted = np.sort(x_column)
    threasholds = (x_column_sorted[:-1] + x_column_sorted[1 : ])/2
    for threshold in threasholds:
      weighted_entropy = information_gain(x_column, y, threshold)
      print(f"Threshold {threshold} : {weighted_entropy}")

    if(weighted_entropy < best_gain):
      best_gain = weighted_entropy
      split_feature = feature
      split_threshold = threshold

  return split_feature, split_threshold